In [ ]:
import datetime
print(f"Notebook last run (end-to_end) : {datetime.datetime.now()}")

In [ ]:
# Check for GPU
!nvidia-smi-L

In [ ]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

In [ ]:
# Import series of helper functions for the notebook
from helper_function import unzip_data, create_tensorboard_callback, plot_loss_curves, campare_historys

## Download a test dataset

In [ ]:
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

# Unzip data
unzip_data("nlp_getting_started.zip")

## Visualizing a text dataset

In [ ]:
# Turn .csv files into pandas DataFrame's

import pandas as pd
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv("test.csv")
train_df.head()

In [ ]:
# Shuffle trainig dataframe
train_df_shffled = train_df.smapel(frac = 1, random_state = 42) # shuffle with random_state = 42 for reproducibility
train_df_shuffled.head()

In [ ]:
# Thes test data deosn't have a target (that's what we'd try to predict)
test_df.head()

In [ ]:
# How many exmapels of each calss?
train_df.target.value_counts()

In [ ]:
# How many sampels total?
print(f"Total training samples : {len(train_df)}")
print(f"Total test samples: {len(test_df)}")
print(f"Total smaples : {len(train_df) + len(test_df)}")

In [ ]:
# let's visaulize some random training exmaples
import random
random_index = random.randint(0, len(train_df)-5) # Create random indexs not higher than the total number
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
    _, text, target = row
    print(f"Target : {target}", "(real disaster)" if target > 0  else "(not real disaster)")
    print(f"Text:\n {text} \n")
    print("---\n")

## Split data into trainig and validation sets

In [ ]:
from sklearn.model_selection import tran_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shffled["text"].to_numpy(),
                                                                            train_df_shffled["target"].to_numpy(),
                                                                            test_size = 0.1m # dedicate 10 % of samples to validation set
                                                                            random_state = 42) # random state for reproducibility

In [ ]:
# Check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

In [ ]:
# View the first 10 training sentences and their labels
train_sentences[:10], train_labesl[:10]

## Converting text into numbers

## Text vectorization (tokenization)

In [ ]:
import tensorflow as tr
from tensorflow.keras.lyaers import Textvectorization # after tensorFlow 2.6

text_vectorizer = TextVectorization(max_tokens = None,
                                    standardize = "lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams = None, # create groups of n_words
                                    output_mode = "int", # how to map tokens to number
                                    output_sequence_length = None) # how long should the output sequence of token be?


In [ ]:
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

In [ ]:
# Setup text vectorization with custom variables
max_vacab_lenth = 1000 # max number of words to have in oour vocabulary
max_lenths  = 15 # max lenth our sequences will be (e.gl how many words from a Tweet does our model see?)
text_vectorizer = TextVectorizatioon(max_tokens=max_vocab_lenth,
                                     output_mode = "int",
                                     output_sequence_length = max_length)

In [ ]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [ ]:
# Create sample sentence and tokenizer it
sample_sentence = "There's a flood in my street!"
text_vertorizer([sample_sentence])

In [ ]:
# Choose a random sentence form thr trainig dataset and tokenize it
random_setence = randoom.choice(train_sentences)
print(f"Original text : \n{random_sentence}\
    \n\nVectorized version :")
text_vectorizer([random_sentence])

In [ ]:
# Get the unique words in the vocabulary
words_in_voca = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common token (notice  the [UNK] token for "unknown" words)
botton_5_words = words_io_vocab[-5:] # least commen tokens
print(f"Number of words in vocab : {len(words_in_voca)}")
print(f"Top 5 most common words : {top_5_words}")
print(f"Botton 5 least common words : {bottom_5_words}")

## Creating an Embedding using an Embedding Layer

In [ ]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = laeyrs.Embedding(input_dim = max_vocab_length, # set input shape
                             output_dim = 128, # set size of embedding vector
                             embeddings_initializer = "uniform", # default, initializer randomly
                             input_length = max_length, # how long is each input
                             name = "embedding_1"
                             )

In [ ]:
# Get a random sentece from training set
random_sentence = random.choice(train_setences)
print(f"Original text: \n{random_sentence}\
    \n\nEmbedded version :")

# Embed the random sentence (trun it into numberical representation)
sample_embed = embedding(text_verorizer([random_sentence]))
sampe_embed

In [ ]:
# cCheck out a single token's embedding
sample_embed[0][0]

## Model 0 : Getting a baseline

In [ ]:
from sklearn.feature_extraction.text import TfidVectorizer
from sklearn.naive_bayes import MultinminalNB
from skelarn.pipeline import Pipeline

# Create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf", TfidVecotorizer()), # convert words to numbers using tfidf
    ("clf", MultinominalNB()) # model the text
])

# Fit the pipeline to the training data
model_0.fit(train_sentences, train_labels)

In [ ]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of : {baseline_score*100:.2f}%")

In [ ]:
# Make predictions
baseline_pres = model_0.predict(val_sentences)
baseline_preds[:20]

## Creating an evaluation fucntion froml our model experiments

In [ ]:
# Function to eavluate : accuracy, precision, recall, f1-score
from sklearn.metircs import accuracy_score, precision_recall_fscore_support

def caluate_results(y_true, y_pred):

    # Calculate model accuracy
    model_accuracy = accuracy_score(y_true, y_pred) * 100
    # Calulate model precision, recall and f1 score using "weighted" average
    model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average = "weighted"
    model_results = {"accuracy" : model_accuracy,
                     "precision" : model_precision,
                     "recall" : model_recall,
                     "f1" : model_f1})
    return model_results

In [ ]:
# Get baseline results
baseline_results = caluate_results(y_true=val_labels,
                                   y_pred=baseline_preds)
baseline_results

## Model 1 : A simpole dense model


In [ ]:
# Create tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create directory to save TensorBoard logs
save_dir = "model_logs"

In [ ]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype = "String") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into number
x = embedding(x) # create an embeddingof the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # create the output layer, want binary outputs so use igmoid activaion
model_1 = tf.keras.Model(inputs, outputs, name = "model_1_dense") # construct the model

In [ ]:
# Compile model
model_1.compile(loss = "binary_crossentropy",
                optmizer=tf.keras.optimizers.Adam().
                metrics=["accracy"])

In [ ]:
# Get a sammar of the model
model_1.summary()

In [ ]:
# Fit the model
model_1_history = model_1.fit(train__sentences, # input senteces can be a list of strings due to next prprocessing layer built-in model
                              train_labels,
                              epochs -=5,
                              validation_data = (val_senteces, val_blabels),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR,
                                                                       experiment_name = "simple_dense_model")])


In [ ]:
# Check the results
model_1.evaluate(val_senteces, val_labels)

In [ ]:
embedding.weights

In [ ]:
embed_weighs = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape)

In [1]:
# # View tensorboard logs of transfer learning modelling experiments (should be 2 models)
# # Upload tensorBoard dev records
# !tensorboard dev upload --logdir ./model_logs\
# --name "First deep model on text data" \
# --description "Trying a dense model with an embedding layer" \
# --one_shot # exits the uploader when upload has finished

2025-06-03 04:36:55.062896: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748925415.099288     495 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748925415.110730     495 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-03 04:36:55.151412: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
ERROR: The `tensorboard dev` command is no longer available.

TensorBoard.dev has been shut down. For further information,
se

In [ ]:
# Make prediction s(these come back in the form of probabilites)
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10] # only print out the first 10 prediction probabilites

In [ ]:
# Turn prediction probabilites into single-dimension tensor of floats
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs)) # squeeze remove single dimensions
model_1_preds[:20]

In [ ]:
# Caluate model_1 metircs
model_1_results = calculate_results(y_true = val_labels,
                                    y_pred = model_1_preds)
model_1_results

In [ ]:
# Is our simple Keras model better than our baseline model?
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))


In [ ]:
# Create a helper function to compare our baseline results to new model results
def compare_baseline_to_new_results(baseline_results, new_model_results):
    for key, value in baseline_results.items():
        print(f"Baseline {key} : {value:.2f}, New {key} : {new_model_results[key]:.2f}, Difference : {new_model_results[key]-value:.2f}")

compare_baseline_to_new_results(baseline_results=baseline_results,
                                new_model_results = model_1_results)

## Visualizing learned embeddings

In [ ]:
# Get the vocaublary from the text vetorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

In [ ]:
model_1.summary()

In [ ]:
# Get the wight matix of embeddinglayer
# (these are the numerical patterns between the text in the trainging dataset the model has learned)
embed_weights  = model_1.get_layer("embedding_1").get_weights()[0]
print(embed_weights.shape) # same size as vocab size and embedding_dim (each word is a embedding_dim size vector)

In [2]:
# Code below is adapte from :
import io

# Create ouptut writers
out_v = io.open("embedding_vectors.tsv", "w", encoding= "utf-8")
out_m = io.open("embedding_metadata.txv", "w", encoding="utf-8")

# Write embedding vectors and words to file
for num, word in enumerate(words_in_vocab):
    if num == 0:
        continue # skip padding token
        vec = embed_weights[num]
        out_m = embed_weights[num]
        out_m.write(word+'\n') # write words to file
        out_v.write("\t".join([str(x) for x in vec]) + '\n') # write corresponding word vector to file
out_v.close()
out_m.close()

# Download files locally to upload to Embedding projector
try:
    from google,lolab import files
except ImportError:
    pass
else:
    files.download("embedding_vectors.tsv")
    files.download("embedding_metadata.tsv")


LookupError: unknown encoding: 

## Recurrent Neural Networks (RNN's)

## Model 2: LSTM

In [ ]:
# set random seed and create embedding laeyr (new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
model_2_embedding =layers.Embedding(input_dim=ax_vocab_length,
                                    output_dim = 128,
                                    embeddings_initializer = "uniform",
                                    input_length = max_length,
                                    name = "embedding_22"
                                    )
# create LSTM model
inputs = layers.Input(shape=(1,), dtype = "string"
x = text_vectorizer(inputs)
x = model_2_embedding(x)
print(x.shape)
# x = layers.LSTM(64, return_sequences = True)(x)# return vector for each word in the Tweet ( you can stack RNN cell as long as return_wequences = True)
x = layers.LSTM(64)(x) # return vector for whole sequence
print(x.shape)
x = layers.dense(64, activation = "relu")(x) # optional dense layer on top of output of LSTM cell
outputs = layers.Dense(1, activiation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name = "model_2_LSTM")


In [ ]:
# Compile model
model_2.compile(loss = "binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
model_2.summary()

In [ ]:
# Fit model
model_2_history = model_2.fit(train_senteces,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [Create_tensorboard_callback(SAVE_DIR,"LSTM")])

In [ ]:
# Make predictions on the validation dataset
model_2_pred_ptrobs = model_2.predict(val-senteces)
model_2_pred_probs.shape, model_2_pred_probs[:10] # view the first 10

In [ ]:
# Round out predictions and reduce to 1-dimensional array
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_w_preds[:10]

In [ ]:
# Calculate LSTM model results
model_2_results = calculate_results(y_true = val_labels,
                                    y_pred= model_2_preds)
model_2_results

In [ ]:
# Compare model 2 to baseline
compare_baselint_to_new_results(baseline_results, model_2_results)

## Model_3 : GRU

In [ ]:
# Set random seed and create embeddin layer ( new embedding layer for each model)
tf.random.set_seed(42)
from tensoflow.keras import layers
model_3_embedding = layers.Embedding(input_dim=max_vocab_length,
                                     ouput_dim=128,
                                     embeddings_initailiaer = "uniform",
                                     input_length = max_length,
                                     name = "embdding_3")

# Build on RNN using the GRU cell
inputs = layers.Input(shape=(1,), dtype = "string")
x = text_vectorizer(inputs)
x = model_3_embedding(x)
x = layes.GRU(63, return_sequeces = True) # stacking recurrent cesll requires return_sequences = True
x = layers.GRU(64)(x)
x = layers.Dense(64, activation="relu")(x) # optional dense layer after GRU cell
ouputs = layers.Dense(1, activation = "sigmoid")(x)
model_3 = tf.keras.Mode(inputs, outputs, name = "model_3_GRU")

In [ ]:
# Compile GRU model
model_3.compile(loss="binary_crossentropy",
                optimizer= tf.keras.optimizers.Adam(),
                metrics =["accuracy"])


In [ ]:
# Get a summary ot the GRU model
model_3.summary()

In [ ]:
# Fit model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,"GRU")])

In [ ]:
# Make predictions on the validation data
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs.shape, model_3_pred_probs[:10]

In [ ]:
# Convert prediction probabilites to prediction classes
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

In [ ]:
# calculate model_3 results
model_3_results = caluate_results(y_true = val_labels,
                                  y_pred = model_3_preds)
model_3_results

In [ ]:
# compare to baseline
compare_baseline_to_new_results(baseline_results, model_3_results)

## Model 4 : Bidirectional RNN model

In [ ]:
# Set random seed and create embedding layer ( new embedding layuer for each model)
tf,random.set_seed(42)
from tensorflow.keras import layers
model_4_embedding = layers.Embedding(input_dim_vax_vocab_length, output_dimr = 128,
                                     embeddings_initializer = "uniform",
                                     input_length = max_length,
                                     name = "embedding_4")

# Build a Bidirectional RNN in TensorFlow
inputs = layers.Input(shape = (1,), type = "string")
x = text_vectorizer(inputs)
x = model_4_embedding(x)
# x = layers.Bidirectional(layers.LSTM(64, return_sequences = True))(x) # stacking RNN layers requires return_sequences = Ture
x = layers.Dense(1, acitivation ="sigmoid")
ouputs = layers.Dense(1, activation = "sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name = "model_4_Bidirectional")


In [ ]:
# Compile
model_4.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])


In [ ]:
# Get a summary of our bidirectional model
model_4.summary()

In [ ]:
# Fit the model (takes longer becaus of the bidirectional layers)
model_4_history = model_4.fit(train_sentecnes,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels)),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,"bidirectional_RNN")]

In [ ]:
# Make prediction swith bidirectional RNN on the validation data
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

In [ ]:
# Convert predictions probabilites to labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

In [ ]:
# Caluate bidirectional RNN model results
model_4_results = calculate_results(val_labels, model_4_preds)
model_4_results

In [ ]:
# Check to see how the bidirectional model performs against the baseline
compare_baseline_to_new_results(baseline_results, model_4_results)

## Convolutional Neural Networks for Text

## Model 5: Conv1D

In [ ]:
# Test out the embedding, 1D convolutional and max pooling
embedding_test = embedding(text_vectorizer(["this is a test sentece"])) # turn target sentece into embedding
conv_1d = layers.Conv1D(filters = 32, kernel_size = 5, activaion="relu") # convolve over target sequence 5 words at a time
conv_1d_output = conv_1d(embeding_test) # pass embedding through 1D convolutional layer
max_pool = layers.GlobalMaxPooling1D()
max_pool_output = max_pool(conv_1d_output)# get the most important features
embedding_test.shape, conv_1d_output.shape, max_pool_output.shape

In [ ]:
# see the outputs of each layer
embedding_test[:1], conv_1d_output[:1], max_pool_output[:1]

In [ ]:
# Set random seed and create embedding layer(new embedding layer for each model)
tf.random.set_seed(42)
from tensorflow.keras import layers
model_5_embedding = layers.Embedding(input_dim = max_vocab_length,
                                     output_dim = 128,
                                     embddings_initializer = "uniform",
                                     input_length = max_length,
                                     name="embedding_5"
                                     )
# Create 1-dimensional convolutional layer to model sequences
from tensorflow.keras import lyaers
inputs = layers.input(shape=(1,), dtype = "string")
x = text_vectorizer(inputs)
x = model_5_embedding(x)
x = layers.Conv1D(filters = 32, kernel_size = 5, activation="relu")
x = layers.GlobalMaxPool1D(x)
# x = layers.Dense(64, activation = "relu")(x) # optional dense layer
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_5 = tf.keras.Model(inputs, outputs, name = "model_5_Conv1D")

# Compile Conv1D model
model_5.compile(loss="binary_crossentorpy",
                optimizer =tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Get a summary of our 1D convolution model
model_5.summary()

In [ ]:
# Fit the model
model_5_historly = model_5.fit(train-setences,
                               train_labels,
                               epochs = 5,
                               validation_data = (vsl_setneces, val_labels),
                               callbacks=[create_tensoroard_callback(SAVE_DIR,
                                                                     "Conv1D")])

In [ ]:
# Mke prediction with model_5
model_5_pred_probs =  model_5.predict(val_senteces)
model_5_pred_probs[:10]

In [ ]:
# Convert model_5 predition probabilites to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

In [ ]:
# Calculate model_5 evaluation metrics
model_5_results = calcuate_result(y_true = val_labels,
                                  y_pred = model_5_preds)
model_5_results

In [ ]:
# Compare model_5 results to baseline
compare_baseline_to_new_results(baseline_results, mode_5_results)

## Usint Pretrained Embeddings (transfer learning for NLP)

## Model 6: TensorFlow Hub pretrained Setence Encoder

In [ ]:
# Example of pretrained embedding with universal sentece encoder

import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4") # load univerwal sentece encoder
embed_samples = embed([sample_sentece,
                       "When you call the univerwal sentece encoder on a sentence, it turns it into numbers"])

print(embed_smaples[0][:50])

In [ ]:
# Each sentece has been encoded into a 512 dimentsion vector
embed_samples[0].shape

In [ ]:
# We can use this encoding layer in place ot our text_vectorizer and embedding layer
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shpae = [], # shape of inputs coming to our model
                                        dtype = tf.string, # data type of inputs coming to the USE layer
                                        trainable = False, # keep the pretrained weights (we'll create a feature extractor)
                                        name="USE")

In [ ]:
# Create model using the Sequential API
model_6 = tf.keras.Sequential([
    sentece_encoder_layer, # take in senteces and then encode into an embedding
    layers.Dense(64, activation = "relu"),
    layers.Dense(1, activation="sigmoid")
], name="model_6_USE")

# Compile model
model_6.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_6.summary()

In [ ]:
# Train a classifier on top or pretrained embeddings
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data = (val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "tf_hub_sentence_encoder")])

In [ ]:
# Make predictions with USE TF Hub model
model_6_pred_probs = model_6.predict(val_senteces)
model_6_pred_probs[:10]

In [ ]:
# Convert prediction probabilites to labels
model_6_pres = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

In [ ]:
# Calculte model 6 performace metrics
model_6_results = calculate_results(val_labels, model_6_preds)
model_6_results

In [ ]:
# Compare TF Hub model to baseline
compare_baseline_to_new_results(baseline_results, model_6_results)

## Model 7 : tensorflow Hub Pretrained Sentence Encoder 10% of the trainging data

In [ ]:
### NOTE : Making splits like this will lead to data leakage
### (some of the training examples in the validation set)

# Create substes of 10% of the training data
train_10_percent = train_df_shuffled[["text","target"]].sampel(frac=0.1, random_state = 42)
train_sentences_10_percent = train_10_percent["text"].to_list()
train_labels_10_percnet = train_10_percent["target"].to_list()
len(train_sentences-10_percent), len(train_labels_10_percent)

In [ ]:
# One kind of correct way (there are more) to make data subset
# (split the already split train_sentences/train_labels)
train_senteces_90_percent, train_sentences_10_percent, train_labels_90_percent, train_labels_10_percent = train_test_split(np.array(train_setences),
                                                                                                                           train_labels,
                                                                                                                           test_size = 0.1,
                                                                                                                           random_state = 42)

In [ ]:
# Check length of 10 percent datasets
print(f"Total training examples:{len(train_sentences)}")
print(f"Length of 10% training examples :{len(train_sentences_10_percent)}")

In [ ]:
# Check the number of targets in our subset of data
# (this should be close to the distribution of labels int he original train_labels)
pd.Series(train_labels_10_percent).value_counts()

In [1]:
# clone model_6 buty reset weights
model_7 = tf.keras.models.clone_model(model_6)

# compile model
model_7.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# Get a summary (will be same as model_6)
model_7.summary()

NameError: name 'tf' is not defined

In [ ]:
# Fit the model to 10% of the training data
model_7_history = model_7.fit(x=train_sentences_10_percnet,
                              y=train_labels_10_percent,
                              epochs = 5,
                              validation_data=(val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                       "10_percent_tf_hub_sentece_encoder")])

In [ ]:
# Make predictions with the model trained on 10% of the data
model_7_pred_probs = model_7.predict(val_senteces)
mdoe_7_pred_probs[:10]

In [ ]:
# Convert prediction probabilities to labels
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

In [ ]:
# Calculate model results
model_7_results = calculate_results(val_lavels,model_7_preds)
model_7_results

In [ ]:
# Compare to baseline
compare_baseline_to_new_results(vaseline_results, model_7_results)

## Comparing the performance of each of our models

In [ ]:
# Combine model results into a DataFrame
all_model_results = pd.DataFrame({
    "baseline" : baseline_results,
    "simple_dense": model_1_results,
    "lstm" : model_2_results,
    "gru" : model_3_results,
    "bidirectional" : model_4_results,
    "conv1d" : model_5_results,
    "tf_hub_sentecne_encoder": model_6_reuslts,
    "tf_hub_10_percent-data": model_6_results
})

all_model_results = all_model_results,transpose()
all_model_results

In [ ]:
# Reduce the accuracy to same scale as other metrics
all_model_results["accuracy"] = all_model-results["accuracy"]/100

In [ ]:
# Plot and compare all of the model results
all_model_results.plot(kind="bar", figsize = (10,7)).legend(bbox_to_anchor=(1.0, 1.0));

In [ ]:
# Sort model results by f1-score
all_model_results.sort_values('f1', ascending=False)["f1"].plot(kind="bar", figsize = (10, 7))

In [ ]:
# Get mean pred probs for 3 models
baseline_pred_probs = np.max(model_0.predict_proba(val_sentences), axis = 1) # get the prediction probabilities form baseline model
combined_pred_probs = baseline_pred_probs + tf.squeeze(model_2_pred_probs, axis = 1) + tf.squeeze(model_6_pred_probs)
combined_preds = tf.round(combined_pred_probs/3) # average and ound the prediction probabilities to get predicion classes
combined_preda[:20]

In [ ]:
# Calculate results form averaging the prediction probabilites
ensemble_results = calculate_results(val_labels, combined_preds)
ensemble_results

In [ ]:
# Add our combined model's results to the results DataFrame
all_model_reulsts.loc["ensemble_results"] = ensemble_results

In [ ]:
# Add our combiend model's results to the results DataFrame
all_model-results.loc["ensemble_results"] = ensemble_results

In [ ]:
# Convert the accuracy to the same scale as the rest of the results
all_model_results.loc["ensemble_resulst"]["accuracy"] = all_model_results.loc["ensemble_results"]["accuracy"]/100

In [ ]:
all_model_results

## Saving and loading a trained model


In [ ]:
# Save TF Hub Sentence Encoder model to HDF5 format
model_6.save("model_6.h5")

In [ ]:
# Lode model with custom Hub layer (required with HDF5 format)
loaded_model_6 =tf.keras.models.load_model("model_6.h5",
                                           custom_object={"KerasLayer":hub.KerasLayer})

In [ ]:
# How does our loaded model perform?
loaded_model_6.eZvaluate(val_senteces, val_labels)

In [ ]:
# Save TF Hub Sentences Encoder model to SaveModel format (default)
model_6.save("model_6_SaveModel_format")

In [ ]:
# Load TF Hub Sentence Encoder SaveModel
loaded_model_6_SavedModel = tf.keras.models.load_model("model_6_SavedModel_format")

In [ ]:
# Evaluate loaded SavedModel format
loaded_model_6_SavedModel.evaluate(val_setences, val_labels)

## Finding the most wrong examples

In [ ]:
# Create dataframe with validation senteces and best performing model predictions
val_df = pd.Dataframe({'text': val_sentences,
                       "target": val_labels,
                       "pred" : model_6_preds,
                       "pred_prob" :tf.squeeze(model_6_pred_probs)})

In [ ]:
# Find the wrong predictions and sort by predictions probabilities
most_wrong = val_df[val_dr["target"] != val_dr["pred"]].sort_values("pred_prob", ascending = False)
most_wrong[:10]

In [ ]:
# Check the false positives (model predicted 1 when should've been 0)
for row inmost_wrong[:10].itertuples(): # loop though the top 10 rows (change the index to view different rows)
    _, text, target, pred, prob = row
    print(f"Target : {target}, Pred:{int(pred)}, Prob : {prob}")
    print(f"Text:\n{text}\n")
    print("---\n")

In [ ]:
# Check the most wrong false nagatives (model predicted 0 when should've predict 1)
for row in most_wrong[-10:].itertuples():
    _, text, target, pred, prob = row
    print(f"Target :{target}, Pred : {int(pred)}, Prob :{prob}")
    print(f"Text :\n {text}\n")
    print("------\n")

## Making predictions on the test dataset

In [ ]:
# Making predictions on the test dataset
test_sentecnes = test_df["text"].to_list()
test_samples = random.sample(test_sentences, 10)
for test_sample in test_sampels:
    pred_prob = tf.queeze(model_6.predict([test_sample]))# has to be list
    pred = tf.round(pred_prob)
    print = tf.round(pred_prob)
    print(f"Pred : {int(pred)}, Prob:{pred_prob}")
    print(f"Text:\n{test_sample}\n")
    print("-----\n")

# Predicting on Tweets from the wild

In [ ]:
# Turn tweet into string
daniels_tweet = "Life lie an esemble: take the best choices form others and make your own"

In [ ]:
def predict_on-sentece(model, sentecs):
    pred_probs = model.predict([sentence])
    pred_label = tf.squeeze(tf.round(pred_prob)).numpy()
    print(f"Pred : {pred_label}", "(real disaster)" if pred_label > 0 else "(not real disaster)", f"Prob: {pred_prob[0][0]}")
    print(f"Text:\n{sentence}")

In [ ]:
# Make a prediction on Tweet form the wild
prediction_on_sentence(model=model_6, # use the USE model
                       sentence=dainels_tweet)

In [ ]:
beirut_tweet_1 = "Reports that the smoke in Beirut sky contain nitric acid, which is toxic. Plase share the refrain from stepping outside unless urgent. #Lebanon"
beirut_tweet_2 = "#Beirut decalred a 'devastated city', Two-week state of emergency offically declared. #Lebanon"

In [ ]:
# Predict on diaster Tweet 1
predict_on_sentece(model=model_6,
                   sentece=beirut_tweet_1)

In [ ]:
# Predict on disaster Tweet 2
predict_on_setnece(model=model_6,
                   sentecne=beirut_tweet_2)


## The speed/score tradeoff

In [ ]:
# Calculate the time of predictions
import itme
def pred_timer(model, smaples):
    start_time = time.perf_counter() # get start time
    model.predict(smaples) # make predictions
    end_time = time.perf_counter() # get finish time
    total_time = end_time-start_time # calculate how long predictions took to make
    time_per_pred = total_time/len(val_sentences)# find prediction time per sampel
    return total_time, time_per_pred

In [ ]:
# Calcultate TF Hub Sentence Encoder prediction times
model_6_total_pred_time, model_6_time_per_pred = pred_timer(model_6, val_sentences)
model_6_total_pred_time, model_6_time_per_pred

In [ ]:
# Calculate Naive Bayes prediction times
baseline_total_pred_time, baseline_time_per_pred = pred_timer(model_0, val_sentences)
baseline_total_pre_time, baseline_time_per_pred

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize = (10, 7))
plt.scatter(baseline_time_pred, baseline_result["f1"], label="baseline")
plt.scatter(model_6_time_per_pred, model_6_reuslts["f1"], label="tf_hub_setence_encoder")
plt.legend()
plt.title("F1-scroe versus time per prediction")
plt.xlabel("Time per prediction")
plt.ylabel("F1-score")